In [1]:
from ase.db import connect
from ase import Atoms
from ase.calculators.emt import EMT
import numpy as np
import pandas as pd

from ase.visualize import view
from ase.io import read


In [2]:
formed_data_path = "/media/mohammed/Work/FORMED_ML/Data_FORMED.csv"
formed_data = pd.read_csv(formed_data_path)
for column in formed_data.columns:
    print(column)


name
S1_exc
S2_exc
S3_exc
S4_exc
S5_exc
T1_exc
T2_exc
T3_exc
T4_exc
T5_exc
HOMO
LUMO
gap
S1_ehdist
S2_ehdist
S3_ehdist
S4_ehdist
S5_ehdist
T1_ehdist
T2_ehdist
T3_ehdist
T4_ehdist
T5_ehdist
S1_osc
S2_osc
S3_osc
S4_osc
S5_osc
natoms
pentavalent_As/As(V)
trivalent/As(III)
monovalent/As(I)
other_As
borane_(BR3)
borinate_(BOR2,inc._borinic_esters_and_acids)
boronate_(BO2R,inc._boronic_esters_and_acids)
borate_(BO3)
diborane_(B-B)
boron_cluster/(poly)borane_(B-B-B)
tetracoordinated_boron_(BR4)
alkane_(R3C-CR3)
alkene_(R2C=CR2)
alkyne_(RC#C3)
allene_(R2C=C=R2)
cumulene_(C=C=C=C)
amide_(C(=O)N)
enamine_(R2C=CNR)
primary_amine_(NRH2)
secondary_amine_(NR2H)
tertiary_amine_(NR3)
azo_(N=N,not_including_azides)
hydrazine_(N-N)
iminium_(R2C=NR2+)
nitrile_(RC#N)
tetracoordinated_nitrogen_(NR4)
isonitrile_(RN#C)
azide_(N=N=N)
amine-borane_(N-B)
imine_(C=N)
alcohol_(CR3-OH)
ketone_(C(C=O)C)
aldehyde_(CHO)
carboxylate_(RCOOR,acid_and_ester)
carboxylate_(RCOO-,base)
peroxide_(ROOR)
ether_(C-O-C)
alkoxide

In [3]:
names = np.load("/media/mohammed/Work/FORMED_ML/data/names.npy")
representations = np.load("/media/mohammed/Work/FORMED_ML/data/repr.npy")
dict_data_org = dict(zip(names,representations))
dict_data ={}
for name  in dict_data_org.keys():
    dict_data[name.split("/")[-1].split(".")[0]] = dict_data_org[name]
print(name.split("/")[-1].split(".")[0])
#df_representation = pd.DataFrame.from_records(dict_data,orient='index')

VACHIS


In [4]:
db = connect('formed.db')
mol_name = "VACHIS"
atoms = read(f"/media/mohammed/Work/FORMED_ML/XYZ_FORMED/{mol_name}.xyz")
data_extra = formed_data[formed_data["name"] == mol_name].iloc[0].to_dict()
data_extra["slatm"] = dict_data[mol_name]
db.write(atoms, data = data_extra, name = mol_name)

116688

In [6]:
def add_atoms_to_db_from_df(db, mol_name, df):
    atoms = read(f"/media/mohammed/Work/FORMED_ML/XYZ_FORMED/{mol_name}.xyz")
    data_extra = df[df["name"] == mol_name].iloc[0].to_dict()
    data_extra["slatm"] = dict_data[mol_name]
    db.write(atoms, data = data_extra, name = mol_name)
db = connect('formed.db')
formed_data_sample = formed_data
for mol_name in formed_data_sample["name"]:
    add_atoms_to_db_from_df(db, mol_name, formed_data_sample)
    #print(mol_name)


## split the data

In [28]:
#random split of the dataset
import os
os.remove('val/formed_val.db')
os.remove('formed_test.db')
os.remove('train/formed_train.db')
db_val = connect('val/formed_val.db')
db_test = connect('formed_test.db')
db_train = connect('train/formed_train.db')
# remove the previous datasets

# split the dataset
db = connect('formed.db')
with connect('formed.db') as db:
    for row in db.select():
        if np.random.rand() < 0.1:
            db_val.write(row)
        elif np.random.rand() < 0.1:
            db_test.write(row)
        else:
            db_train.write(row)




## add property to database

In [25]:
with connect('formed.db') as db:
    for row in db.select():
        data = row.data
        data["log_S1_osc"] = np.log(data["S1_osc"]+1e-8)
        db.update(row.id, data=data)
    row = db.get(row.id)
print(row.data["log_S1_osc"])

-4.1997044112134825
